In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

# Importing Deep Learning Libraries

from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

In [3]:
pic_size = 48
folder_path = "../input/face-expression-recognition-dataset/images/"

In [4]:
expression = 'happy'

plt.figure(figsize = (12,12))
for i in range(1,10,1):
    plt.subplot(3,3,i)
    img = load_img(folder_path+"train/"+expression+"/"+
                  os.listdir(folder_path + "train/" + expression)[i], target_size = (pic_size, pic_size))

    plt.imshow(img)
plt.show()

In [5]:
batch_size = 128

datagen_train = ImageDataGenerator()
datagen_val = ImageDataGenerator()

Train_set = datagen_train.flow_from_directory(folder_path+"train",target_size = (pic_size, pic_size), color_mode = "grayscale", batch_size = batch_size, class_mode = 'categorical', shuffle = True)

Test_set = datagen_val.flow_from_directory(folder_path+"validation", target_size = (pic_size, pic_size),color_mode = "grayscale", batch_size = batch_size, class_mode = "categorical", shuffle = False)

In [6]:
No_of_classes = 7

model = Sequential()
# 1st model CNN
model.add(Conv2D(64,(3,3),padding = 'same', input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd model CNN
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
#3rd model CNN
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
#4th model CNN
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
# packing layers
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(No_of_classes, activation = 'softmax'))

op = Adam(lr = 0.0001)
model.compile(optimizer=op, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [16]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
check_point = ModelCheckpoint("./model.h5", monitor = 'val.acc', verbose = 1, save_best_only = True, mode = 'max')

early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3, verbose = 1, restore_best_weights = True)

reduce_learning_rate = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)


callbacks_list = [early_stopping, check_point, reduce_learning_rate]

epochs = 50

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr=0.001), metrics = ['accuracy'])

In [8]:
history = model.fit_generator(generator=Train_set,
                             steps_per_epoch=Train_set.n//Train_set.batch_size,
                             epochs=epochs,
                             validation_data=Test_set,
                             validation_steps=Test_set.n//Test_set.batch_size,
                             callbacks=callbacks_list)

In [15]:
plt.style.use('dark_background')
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize = 20)
plt.xlabel('Epoch', fontsize = 20)
plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.legend(loc = 'upper right')

plt.subplot(1,2,2)
plt.ylabel('Accuracy', fontsize = 20)
plt.xlabel('Epoch', fontsize = 20)
plt.plot(history.history['accuracy'], label = 'Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.show()